# Multidimensional data frames: Using PySpark with JSON data

Thus far, we have used PySpark’s data frame to work with textual and tabular data.
Let's push the abstraction a little further by representing *hierarchical information* within a data frame. Imagine it for a moment: columns within columns, the ultimate flexibility.

## Reading JSON data

This section explains what JSON is, how to use the specialized JSON reader with
PySpark, and how a JSON file is represented within a data frame. For this chapter, we use a JSON dump of information about the TV show Silicon Valley from TV Maze.

```json
{
   "id":143,
   "name":"Silicon Valley",
   "type":"Scripted",
   "language":"English",
   "genres":[
      "Comedy"
   ],
   "network":{
      "id":8,
      "name":"HBO",
      "country":{
         "name":"United States",
         "code":"US",
         "timezone":"America/New_York"
      }
   },
   "_embedded":{
      "episodes":[
         {
            "id":10897,
            "name":"Minimum Viable Product",
            "season":1,
            "number":1
         },
         {
            "id":10898,
            "name":"The Cap Table",
            "season":1,
            "number":2
         }
      ]
   }
}
```

In [18]:
# Reading a simple JSON document as python dictionary
import json
sample_json = """{
   "id":143,
   "name":"Silicon Valley",
   "type":"Scripted",
   "language":"English",
   "genres":[
      "Comedy"
   ],
   "network":{
      "id":8,
      "name":"HBO",
      "country":{
         "name":"United States",
         "code":"US",
         "timezone":"America/New_York"
      }
   }
}"""
document = json.loads(sample_json)
print(document)
print(type(document))

{'id': 143, 'name': 'Silicon Valley', 'type': 'Scripted', 'language': 'English', 'genres': ['Comedy'], 'network': {'id': 8, 'name': 'HBO', 'country': {'name': 'United States', 'code': 'US', 'timezone': 'America/New_York'}}}
<class 'dict'>


In [19]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os

spark = SparkSession.builder.getOrCreate()

DIRECTORY = "data/shows/"

shows = spark.read.json(
    os.path.join(DIRECTORY,'shows-silicon-valley.json')
)

shows.count()

1

In the PySpark world, reading JSON follows this rule: *one JSON document, one line, one record*. This means that if you want to have multiple JSON records in the same document, you need to have one document per line and no new line within your document.

If you want to ingest multiple documents across multiple files, you need to set
the `multiLine` (careful about the capital L!) parameter to `true`. This will changethe JSON reading rule to the following: *one JSON document, one file, one record*.

In [20]:
three_shows = spark.read.json(
    os.path.join(DIRECTORY,"shows-*.json"),
    multiLine=True)
    
three_shows.count()

3

### Complex data types
ARRAY, MAP, STRUCT

Spark uses this the term *complex data types* to refer to data types that contain other types.  In PySpark, we have the *array*, the *map*, and the *struct*. With these, you will be able to express an infinite amount of data layout.

In [21]:
shows.printSchema()

root
 |-- _embedded: struct (nullable = true)
 |    |-- episodes: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _links: struct (nullable = true)
 |    |    |    |    |-- self: struct (nullable = true)
 |    |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- airdate: string (nullable = true)
 |    |    |    |-- airstamp: string (nullable = true)
 |    |    |    |-- airtime: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- image: struct (nullable = true)
 |    |    |    |    |-- medium: string (nullable = true)
 |    |    |    |    |-- original: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- number: long (nullable = true)
 |    |    |    |-- runtime: long (nullable = true)
 |    |    |    |-- season: long (nullable = true)
 |    |    |    |-- summary: string (nullable = true)
 |    |    |    |-- url: string (nullable = true

In [22]:
# the hierarchy within the schema. 
# PySpark took every top-level key—the keys 
# from the root object—and parsed them as columns 

print(shows.columns)

['_embedded', '_links', 'externals', 'genres', 'id', 'image', 'language', 'name', 'network', 'officialSite', 'premiered', 'rating', 'runtime', 'schedule', 'status', 'summary', 'type', 'updated', 'url', 'webChannel', 'weight']


In [23]:
array_subset = shows.select("name","genres")

array_subset.show(1,False)

+--------------+--------+
|name          |genres  |
+--------------+--------+
|Silicon Valley|[Comedy]|
+--------------+--------+



our genres column can be thought of as containing lists of elements within each record. To get to the value inside the array, we need to extract them. PySpark provides a very pythonic way to work with arrays as if they were lists.

PySpark’s array functions—available in the pyspark.sql.functions module—are
almost all prefixed with the `array_` keyword

Let's play with these functions.

- We create three literal columns (using `lit()` to create scalar columns, then
`make_array()`) to create an array of possible genres. PySpark won’t accept
Python lists as an argument to `lit()`, so we have to go the long route by creating individual scalar columns before combining them into a single array.
- Then use the function `array_repeat()` to create a column repeating the Comedy string we extracted below five times. Finally compute the size of both
columns, de-dupe both arrays, and intersect them, yielding our original [Comedy] array

In [24]:
array_subset = array_subset.select(
    "name",
    array_subset.genres[0].alias("dot_and_index"),
    F.col("genres")[0].alias("col_and_index"),
    array_subset.genres.getItem(0).alias("dot_and_method"),
    F.col("genres").getItem(0).alias("col_and_method"),
)
array_subset.show()

+--------------+-------------+-------------+--------------+--------------+
|          name|dot_and_index|col_and_index|dot_and_method|col_and_method|
+--------------+-------------+-------------+--------------+--------------+
|Silicon Valley|       Comedy|       Comedy|        Comedy|        Comedy|
+--------------+-------------+-------------+--------------+--------------+



In [25]:
array_subset_repeated = array_subset.select(
    "name",
    F.lit("Comedy").alias("one"),
    F.lit("Horror").alias("two"),
    F.lit("Drama").alias("three"),
    F.col("dot_and_index"),
).select(
    "name",
    F.array("one", "two", "three").alias("Some_Genres"),
    F.array_repeat("dot_and_index", 5).alias("Repeated_Genres"),
)

array_subset_repeated.show(1, False)

+--------------+-----------------------+----------------------------------------+
|name          |Some_Genres            |Repeated_Genres                         |
+--------------+-----------------------+----------------------------------------+
|Silicon Valley|[Comedy, Horror, Drama]|[Comedy, Comedy, Comedy, Comedy, Comedy]|
+--------------+-----------------------+----------------------------------------+



In [26]:
array_subset_repeated.select(
    "name", F.size("Some_Genres"), F.size("Repeated_Genres")
).show()

+--------------+-----------------+---------------------+
|          name|size(Some_Genres)|size(Repeated_Genres)|
+--------------+-----------------+---------------------+
|Silicon Valley|                3|                    5|
+--------------+-----------------+---------------------+



In [27]:
array_subset_repeated.select(
    "name",
    F.array_distinct("Some_Genres"),
    F.array_distinct("Repeated_Genres"),
).show(1, False)

+--------------+---------------------------+-------------------------------+
|name          |array_distinct(Some_Genres)|array_distinct(Repeated_Genres)|
+--------------+---------------------------+-------------------------------+
|Silicon Valley|[Comedy, Horror, Drama]    |[Comedy]                       |
+--------------+---------------------------+-------------------------------+



In [28]:
array_subset_repeated = array_subset_repeated.select(
    "name",
    F.array_intersect("Some_Genres", "Repeated_Genres").alias("Genres"),
)
array_subset_repeated.show()

+--------------+--------+
|          name|  Genres|
+--------------+--------+
|Silicon Valley|[Comedy]|
+--------------+--------+



When you want to know the position of a value in an array, you can use `array_position()`. This function takes two arguments:
- An array column to perform the search
- A value to search for within the array

In [29]:
array_subset_repeated.select(
    "Genres", F.array_position("Genres", "Comedy")
).show()

+--------+------------------------------+
|  Genres|array_position(Genres, Comedy)|
+--------+------------------------------+
|[Comedy]|                             1|
+--------+------------------------------+



#### MAP type: keys and values within a column

A map is conceptually very close to a Python typed dictionary: you have keys and values just like in a dictionary, but as with the array, the keys need to be of the same type, and the values need to be of the same type (the type for the keys can be different than the type for the values). Values can be null, but keys can’t, just like with Python dictionaries.

One of the easiest ways to create a map is from two columns of type array. We will
do so by collecting some information about the name, language, type, and url columns into an array and using the map_from_arrays() function, like below

In [31]:
columns = ["name", "language", "type"]

shows_map = shows.select(
    *[F.lit(column) for column in columns],
    F.array(*columns).alias("values"),
)
shows_map = shows_map.select(F.array(*columns).alias("keys"), "values")
shows_map.show(1,False)

+----------------------+-----------------------------------+
|keys                  |values                             |
+----------------------+-----------------------------------+
|[name, language, type]|[Silicon Valley, English, Scripted]|
+----------------------+-----------------------------------+



In [32]:
shows_map = shows_map.select(
    F.map_from_arrays("keys", "values").alias("mapped")
)

shows_map.printSchema()

root
 |-- mapped: map (nullable = false)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [33]:
shows_map.show(1,False)

+---------------------------------------------------------------+
|mapped                                                         |
+---------------------------------------------------------------+
|{name -> Silicon Valley, language -> English, type -> Scripted}|
+---------------------------------------------------------------+



In [34]:
# To access the mapped values
shows_map.select(
    F.col("mapped.name"),
    F.col("mapped")["name"],
    shows_map.mapped["name"],
).show()

+--------------+--------------+--------------+
|          name|  mapped[name]|  mapped[name]|
+--------------+--------------+--------------+
|Silicon Valley|Silicon Valley|Silicon Valley|
+--------------+--------------+--------------+



Just like with the `array`, PySpark provides a few functions to work with maps under the `pyspark.sql.functions` module. Most of them are prefixed or suffixed with `map`, such as `map_values()` (which creates an array column out of the map values) or `create_map()` (which creates a map from the columns passed as a parameter, alternating between keys and values)

### Struct: Nesting columns within columns

The struct is akin to a JSON object, in the sense that the key or name of each pair is a string and that each record can be of a different type

In [35]:
shows.select("schedule").printSchema()

root
 |-- schedule: struct (nullable = true)
 |    |-- days: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- time: string (nullable = true)



The struct is very different from the `array` and the `map` in that the number of fields and their names are known ahead of time. In our case, the `schedule` struct column is fixed: we know that each record of our data frame will contain that `schedule` struct (or a `null` value, if we want to be pedantic), and within that struct there will be an array of strings, `days`, and a string, `time`. The `array` and the `map` enforce the types of the values, but not their numbers or names. The struct allows for more versatility of types, as long as you name each field and provide the type ahead of time.

we can visualize that schedule is a data frame of two columns (days and time)
trapped within the column.

![struct data type](images/with_json_struct.png)

#### Navigating structs as if they were nested columns
how to extract values from nested structs inside a data frame. PySpark provides the same convenience when working with nested columns as it would for regular columns.


In [36]:
shows.select(F.col("_embedded")).printSchema()

root
 |-- _embedded: struct (nullable = true)
 |    |-- episodes: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- _links: struct (nullable = true)
 |    |    |    |    |-- self: struct (nullable = true)
 |    |    |    |    |    |-- href: string (nullable = true)
 |    |    |    |-- airdate: string (nullable = true)
 |    |    |    |-- airstamp: string (nullable = true)
 |    |    |    |-- airtime: string (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- image: struct (nullable = true)
 |    |    |    |    |-- medium: string (nullable = true)
 |    |    |    |    |-- original: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- number: long (nullable = true)
 |    |    |    |-- runtime: long (nullable = true)
 |    |    |    |-- season: long (nullable = true)
 |    |    |    |-- summary: string (nullable = true)
 |    |    |    |-- url: string (nullable = true

In [37]:
shows_clean = shows.withColumn(
    "episodes", F.col("_embedded.episodes")
).drop("_embedded")

shows_clean.printSchema()

root
 |-- _links: struct (nullable = true)
 |    |-- previousepisode: struct (nullable = true)
 |    |    |-- href: string (nullable = true)
 |    |-- self: struct (nullable = true)
 |    |    |-- href: string (nullable = true)
 |-- externals: struct (nullable = true)
 |    |-- imdb: string (nullable = true)
 |    |-- thetvdb: long (nullable = true)
 |    |-- tvrage: long (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- image: struct (nullable = true)
 |    |-- medium: string (nullable = true)
 |    |-- original: string (nullable = true)
 |-- language: string (nullable = true)
 |-- name: string (nullable = true)
 |-- network: struct (nullable = true)
 |    |-- country: struct (nullable = true)
 |    |    |-- code: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- timezone: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nul

we can refer to individual elements in the array using the index in brackets after
the column reference. What about extracting the names of all the episodes, which are within the episodes array of structs?

In [38]:
episodes_name = shows_clean.select(F.col("episodes.name"))
episodes_name.printSchema()


root
 |-- name: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [39]:
episodes_name.select(F.explode("name").alias("name")).show(3, False)

+-------------------------+
|name                     |
+-------------------------+
|Minimum Viable Product   |
|The Cap Table            |
|Articles of Incorporation|
+-------------------------+
only showing top 3 rows



### Building and using the data frame schema
Let's see how to define and use a schema with a PySpark data frame.We build the schema for our JSON object programmatically and review the out-ofthe-box types  PySpark offers. Being able to use Python structures (serialized as JSON) means that we can manipulate our schemas just like any other data structure; we can reuse our data manipulation tool kit for manipulating our data frame’s metadata. By doing this, we also address the potential slowdown from inferSchema, as we
don’t need Spark to read the data twice (once to infer the schema, once to perform
the read).

#### Using Spark types as the base blocks of a schema


